In [1]:
import os
import zipfile
import urllib

import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
from fiona.crs import from_epsg
from shapely.geometry import Point, MultiPoint

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.ensemble  import RandomForestClassifier as rfc
#from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score

import pylab as pl
import matplotlib.pylab as plt

pl.rcParams['font.size'] = 20
%pylab inline
%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
df = pd.read_csv('output_final/data_final.csv')
df["date"] = pd.to_datetime(df["date"],format='%Y-%m-%d')
print(df.shape)
df.head()

(333768, 5)


,date,zone,pickup,dropoff,tickets
0,2015-01-01,0.0,28.0,48715.0,0.0
1,2015-01-01,1.0,1.0,0.0,10.0
2,2015-01-01,3.0,9.0,0.0,0.0
3,2015-01-01,4.0,411.0,0.0,43.0
4,2015-01-01,6.0,2.0,0.0,22.0


In [3]:
df.tail()

,date,zone,pickup,dropoff,tickets
333763,2018-06-28,199.0,0.0,0.0,37.0
333764,2018-06-29,105.0,0.0,0.0,72.0
333765,2018-06-29,199.0,0.0,0.0,28.0
333766,2018-06-30,105.0,0.0,0.0,14.0
333767,2018-06-30,199.0,0.0,0.0,19.0


In [4]:
df_zone = df.groupby('zone').sum().reset_index()
df_zone.rename(columns = {'zone':'OBJECTID'}, inplace=True)
df_zone

,OBJECTID,pickup,dropoff,tickets
0,0.0,191603.0,150562325.0,0.0
1,1.0,103.0,874470.0,34883.0
2,2.0,363.0,211.0,177889.0
3,3.0,326635.0,220261.0,44411.0
4,4.0,1398963.0,506395.0,69285.0
5,5.0,35202.0,24670.0,79866.0
6,6.0,68870.0,40331.0,404014.0
7,7.0,2481251.0,1273753.0,828.0
8,8.0,14259.0,9436.0,15953.0
9,9.0,179981.0,108402.0,126887.0


# standardize

In [20]:
def standardize(df):
    return (df-df.mean())/df.std()

# Panel Data
Reference: https://medium.com/pew-research-center-decoded/using-fixed-and-random-effects-models-for-panel-data-in-python-a795865736ab

## FEM (Fixed Effect Model) & REM (Random Effects Models)

### example

In [5]:
import numpy as np
import pandas as pd
from linearmodels import PanelOLS
from linearmodels import RandomEffects

In [6]:
from linearmodels.datasets import jobtraining
data = jobtraining.load()
year = pd.Categorical(data.year)
data = data.set_index(["fcode", "year"])
data["year"] = year
data

employ       sales   avgsal  scrap  rework  tothrs  union  grant  \
fcode  year                                                                     
410032 1987   100.0  47000000.0  35000.0    NaN     NaN    12.0      0      0   
       1988   131.0  43000000.0  37000.0    NaN     NaN     8.0      0      0   
       1989   123.0  49000000.0  39000.0    NaN     NaN     8.0      0      0   
410440 1987    12.0   1560000.0  10500.0    NaN     NaN    12.0      0      0   
       1988    13.0   1970000.0  11000.0    NaN     NaN    12.0      0      0   
       1989    14.0   2350000.0  11500.0    NaN     NaN    10.0      0      0   
410495 1987    20.0    750000.0  17680.0    NaN     NaN    50.0      0      0   
       1988    25.0    110000.0  18720.0    NaN     NaN    50.0      0      0   
       1989    24.0    950000.0  19760.0    NaN     NaN    50.0      0      0   
410500 1987   200.0  23700000.0  13729.0    NaN     NaN     0.0      0      0   
       1988   155.0  19700000.0  14287.0    NaN     NaN     0.0      0      0   
       1989    80.0  26000000.0  15758.0    NaN     NaN    24.0      0      0   
410501 1987     NaN   6000000.0      NaN    NaN     NaN     0.0      0      0   
       1988     NaN   8000000.0      NaN    NaN     NaN     0.0      0      0   
       1989     NaN  10000000.0      NaN    NaN     NaN     0.0      0      0   
410509 1987     NaN         NaN      NaN    NaN     NaN     0.0      0      0   
       1988     NaN   2800000.0  18000.0    NaN     NaN     0.0      0      0   
       1989    20.0   3400000.0  18500.0    NaN     NaN     0.0      0      0   
410513 1987    15.0   1413900.0  18465.0    NaN     NaN     0.0      0      0   
       1988    16.0   1869348.0  18972.0    NaN     NaN     0.0      0      0   
       1989    16.0   1857765.0  19589.0    NaN     NaN    20.0      0      0   
410517 1987    24.0         NaN  13000.0    NaN     NaN     0.0      1      0   
       1988    20.0         NaN  12168.0    NaN     NaN     0.0      1      0   
       1989    18.0         NaN  11960.0    NaN     NaN     0.0      1      0   
410518 1987    48.0   2160900.0  14352.0    NaN     NaN    14.0      0      0   
       1988    47.0   2454500.0  16328.0    NaN     NaN    14.0      0      0   
       1989    66.0   3359800.0  17555.0    NaN     NaN    14.0      0      0   
410521 1987    17.0   1453081.0  16454.0    NaN     NaN   150.0      0      0   
       1988    16.0   1546911.0  18529.0    NaN     NaN   100.0      0      0   
       1989    14.0   1672366.0  26428.0    NaN     NaN     0.0      0      0   
...             ...         ...      ...    ...     ...     ...    ...    ...   
419450 1987   175.0  12500000.0   9360.0    NaN     NaN     0.0      0      0   
       1988   225.0  12500000.0  10400.0    NaN     NaN     0.0      0      0   
       1989   275.0  18200000.0  11440.0    NaN     NaN     3.0      0      0   
419459 1987   110.0  15100000.0  21000.0   1.67    0.21    20.0      1      0   
       1988   125.0  16000000.0  21000.0   2.50    0.23    20.0      1      0   
       1989   225.0  28900000.0  21000.0   1.12    0.15    20.0      1      0   
419461 1987    24.0         NaN  14800.0    NaN     NaN     0.0      0      0   
       1988    23.0         NaN  15200.0    NaN     NaN     0.0      0      0   
       1989    36.0         NaN  15800.0    NaN     NaN    40.0      0      1   
419467 1987     4.0   1494764.0  30000.0    NaN     NaN    80.0      0      0   
       1988     5.0   2566104.0  31000.0    NaN     NaN   100.0      0      0   
       1989    10.0   2191680.0  35000.0    NaN     NaN   100.0      0      0   
419472 1987   120.0         NaN  13000.0    NaN     NaN    20.0      0      0   
       1988   100.0         NaN  13500.0    NaN     NaN    20.0      0      0   
       1989   100.0         NaN  14000.0    NaN     NaN    60.0      0      1   
419473 1987    18.0         NaN      NaN    NaN     NaN     0.0      1      0   
       1988    18.0         NaN      NaN    

In [7]:
# random effects model
exog_vars = ["grant", "employ"]
exog = sm.add_constant(data[exog_vars])
mod = RandomEffects(data.clscrap, exog)
re_res = mod.fit()
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:                clscrap   R-squared:                        0.0165
Estimator:              RandomEffects   R-squared (Between):              0.0314
No. Observations:                 105   R-squared (Within):               0.0015
Date:                Sun, Jul 21 2019   R-squared (Overall):              0.0199
Time:                        14:47:22   Log-likelihood                   -77.721
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      0.8542
Entities:                          53   P-value                           0.4286
Avg Obs:                       1.9811   Distribution:                   F(2,102)
Min Obs:                       1.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             0.8634
                            

C:\ProgramData\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [8]:
# fixed effects model
mod = PanelOLS(data.clscrap, exog, entity_effects=True)
re_res = mod.fit()
print(re_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                clscrap   R-squared:                        0.0142
Estimator:                   PanelOLS   R-squared (Between):             -0.0927
No. Observations:                 105   R-squared (Within):               0.0142
Date:                Sun, Jul 21 2019   R-squared (Overall):             -0.0529
Time:                        14:47:22   Log-likelihood                   -40.109
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      0.3590
Entities:                          53   P-value                           0.7001
Avg Obs:                       1.9811   Distribution:                    F(2,50)
Min Obs:                       1.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             0.3590
                            

In [9]:
# Equivalence of fixed effects model and dummy variable regression
data = jobtraining.load()
data["year"] = pd.Categorical(data.year)
FE_ols = smf.ols(formula="clscrap ~ 1 + grant + employ + C(fcode)", data = data).fit()
print(FE_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                clscrap   R-squared:                       0.630
Model:                            OLS   Adj. R-squared:                  0.230
Method:                 Least Squares   F-statistic:                     1.577
Date:                Sun, 21 Jul 2019   Prob (F-statistic):             0.0529
Time:                        14:47:22   Log-Likelihood:                -40.109
No. Observations:                 105   AIC:                             190.2
Df Residuals:                      50   BIC:                             336.2
Df Model:                          54                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.0558      0

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


### Modeling

In [39]:
data = df.copy()[['date', 'zone', 'pickup', 'tickets']]
date = pd.Categorical(data.date)
data = data.set_index(['zone', 'date'])
data['date'] = date
print(data.shape)
data.head()

(333768, 3)


,,pickup,tickets,date
zone,date,,,
0.0,2015-01-01,28.0,0.0,2015-01-01
1.0,2015-01-01,1.0,10.0,2015-01-01
3.0,2015-01-01,9.0,0.0,2015-01-01
4.0,2015-01-01,411.0,43.0,2015-01-01
6.0,2015-01-01,2.0,22.0,2015-01-01


In [40]:
from linearmodels import PanelOLS
from linearmodels import RandomEffects

In [41]:
# Hausman test 

In [42]:
# random effects model
exog_vars = ["pickup"]
exog = sm.add_constant(data[exog_vars])
mod = RandomEffects(data.tickets, exog)
re_res = mod.fit()
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:                tickets   R-squared:                    -6.237e-06
Estimator:              RandomEffects   R-squared (Between):             -0.0001
No. Observations:              333768   R-squared (Within):            5.141e-07
Date:                Sun, Jul 21 2019   R-squared (Overall):             -0.0001
Time:                        14:53:58   Log-likelihood                -1.895e+06
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                     -2.0816
Entities:                         264   P-value                           1.0000
Avg Obs:                       1264.3   Distribution:                F(1,333766)
Min Obs:                       8.0000                                           
Max Obs:                       1277.0   F-statistic (robust):             0.1391
                            

In [43]:
# fixed effects model
mod = PanelOLS(data.tickets, exog, entity_effects=True)
re_res = mod.fit()
print(re_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                tickets   R-squared:                     5.197e-07
Estimator:                   PanelOLS   R-squared (Between):             -0.0002
No. Observations:              333768   R-squared (Within):            5.197e-07
Date:                Sun, Jul 21 2019   R-squared (Overall):          -8.903e-05
Time:                        14:54:00   Log-likelihood                -1.895e+06
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      0.1733
Entities:                         264   P-value                           0.6772
Avg Obs:                       1264.3   Distribution:                F(1,333503)
Min Obs:                       8.0000                                           
Max Obs:                       1277.0   F-statistic (robust):             0.1733
                            

In [44]:
# Equivalence of fixed effects model and dummy variable regression
data = df.copy()[['date', 'zone', 'pickup', 'tickets']]
FE_ols = smf.ols(formula="tickets ~ 1 + pickup + C(zone)", data = data).fit()
print(FE_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                tickets   R-squared:                       0.703
Model:                            OLS   Adj. R-squared:                  0.703
Method:                 Least Squares   F-statistic:                     2989.
Date:                Sun, 21 Jul 2019   Prob (F-statistic):               0.00
Time:                        14:54:15   Log-Likelihood:            -1.8950e+06
No. Observations:              333768   AIC:                         3.790e+06
Df Residuals:                  333503   BIC:                         3.793e+06
Df Model:                         264                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.0112      1.980  

#### standardize

In [45]:
data = df.copy()[['date', 'zone', 'pickup', 'tickets']]
date = pd.Categorical(data.date)
data = data.set_index(['zone', 'date'])
data['date'] = date

data['pickup'] = standardize(data['pickup'])
data['tickets'] = standardize(data['tickets'])

print(data.shape)
data.head()

(333768, 3)


,,pickup,tickets,date
zone,date,,,
0.0,2015-01-01,-0.701916,-0.788801,2015-01-01
1.0,2015-01-01,-0.724372,-0.711717,2015-01-01
3.0,2015-01-01,-0.717719,-0.788801,2015-01-01
4.0,2015-01-01,-0.383375,-0.457342,2015-01-01
6.0,2015-01-01,-0.723541,-0.619217,2015-01-01


In [46]:
# random effects model
exog_vars = ["pickup"]
exog = sm.add_constant(data[exog_vars])
mod = RandomEffects(data.tickets, exog)
re_res = mod.fit()
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:                tickets   R-squared:                     4.784e-07
Estimator:              RandomEffects   R-squared (Between):             -0.0001
No. Observations:              333768   R-squared (Within):            5.141e-07
Date:                Sun, Jul 21 2019   R-squared (Overall):             -0.0001
Time:                        14:54:26   Log-likelihood                -2.712e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      0.1597
Entities:                         264   P-value                           0.6895
Avg Obs:                       1264.3   Distribution:                F(1,333766)
Min Obs:                       8.0000                                           
Max Obs:                       1277.0   F-statistic (robust):             0.1391
                            

In [47]:
# fixed effects model
mod = PanelOLS(data.tickets, exog, entity_effects=True)
re_res = mod.fit()
print(re_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                tickets   R-squared:                     5.197e-07
Estimator:                   PanelOLS   R-squared (Between):             -0.0002
No. Observations:              333768   R-squared (Within):            5.197e-07
Date:                Sun, Jul 21 2019   R-squared (Overall):          -8.903e-05
Time:                        14:54:35   Log-likelihood                 -2.71e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      0.1733
Entities:                         264   P-value                           0.6772
Avg Obs:                       1264.3   Distribution:                F(1,333503)
Min Obs:                       8.0000                                           
Max Obs:                       1277.0   F-statistic (robust):             0.1733
                            

In [49]:
# Equivalence of fixed effects model and dummy variable regression
data = df.copy()[['date', 'zone', 'pickup', 'tickets']]
data['pickup'] = standardize(data['pickup'])
data['tickets'] = standardize(data['tickets'])
FE_ols = smf.ols(formula="tickets ~ 1 + pickup + C(zone)", data = data).fit()
print(FE_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                tickets   R-squared:                       0.703
Model:                            OLS   Adj. R-squared:                  0.703
Method:                 Least Squares   F-statistic:                     2989.
Date:                Sun, 21 Jul 2019   Prob (F-statistic):               0.00
Time:                        15:36:44   Log-Likelihood:            -2.7104e+05
No. Observations:              333768   AIC:                         5.426e+05
Df Residuals:                  333503   BIC:                         5.454e+05
Df Model:                         264                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -0.7892      0.015  

# Spatial: Bayesian network

In [5]:
spatial = pd.read_csv('output_final/spatial.csv').drop(['Unnamed: 0'], axis=1)
# drop taxi zone 1 (Newark Airport) and 104 (Governor's Island/Ellis Island/Liberty Island); optional 263 (Yorkville West); and fill N/A
spatial = spatial.drop([0,103, 104, 105, 262], axis=0).fillna(0)
print(spatial.shape)
spatial.head()

(258, 47)


,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,TotalPop,Men,Women,Hispanic,...,FamilyWork,Unemployment,FELONY,VIOLATION,MISDEMEANOR,subway,bus,sat,meter,parkinglot
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,528.000000,248.750000,279.250000,4.900000,...,0.000000,11.600000,0.0,0.0,0.0,0,4,382.578884,0,1.143448e+05
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,4042.735294,1848.558824,2194.176471,24.332353,...,0.070588,13.126471,79.0,36.0,173.0,3,53,385.980392,16,1.170854e+06
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,4461.681818,2030.363636,2431.318182,37.200000,...,0.218182,8.936364,86.0,72.0,205.0,0,26,431.773308,3,1.891444e+05
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,5654.111111,2756.111111,2898.000000,14.744444,...,0.000000,8.000000,29.0,18.0,42.0,0,38,453.055556,0,3.937056e+05
5,6,0.150491,0.000606,Arrochar/Fort Wadsworth,6,Staten Island,4721.292683,2282.560976,2438.731707,25.720000,...,0.102564,7.615385,46.0,55.0,61.0,1,90,453.055556,2,1.980973e+06


In [6]:
list(spatial.columns)

['OBJECTID',
 'Shape_Leng',
 'Shape_Area',
 'zone',
 'LocationID',
 'borough',
 'TotalPop',
 'Men',
 'Women',
 'Hispanic',
 'White',
 'Black',
 'Native',
 'Asian',
 'Citizen',
 'Income',
 'IncomeErr',
 'IncomePerCap',
 'IncomePerCapErr',
 'Poverty',
 'ChildPoverty',
 'Professional',
 'Service',
 'Office',
 'Construction',
 'Production',
 'Drive',
 'Carpool',
 'Transit',
 'Walk',
 'OtherTransp',
 'WorkAtHome',
 'MeanCommute',
 'Employed',
 'PrivateWork',
 'PublicWork',
 'SelfEmployed',
 'FamilyWork',
 'Unemployment',
 'FELONY',
 'VIOLATION',
 'MISDEMEANOR',
 'subway',
 'bus',
 'sat',
 'meter',
 'parkinglot']

In [7]:
df_spatial = pd.merge(spatial, df_zone, on = 'OBJECTID', how='left')
df_spatial['crime'] = df_spatial['FELONY'] + df_spatial['VIOLATION'] + df_spatial['MISDEMEANOR']
print(df_spatial.shape)
df_spatial.head()

(258, 51)


,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,TotalPop,Men,Women,Hispanic,...,MISDEMEANOR,subway,bus,sat,meter,parkinglot,pickup,dropoff,tickets,crime
0,2,0.433470,0.004866,Jamaica Bay,2,Queens,528.000000,248.750000,279.250000,4.900000,...,0.0,0,4,382.578884,0,1.143448e+05,363.0,211.0,177889.0,0.0
1,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,4042.735294,1848.558824,2194.176471,24.332353,...,173.0,3,53,385.980392,16,1.170854e+06,326635.0,220261.0,44411.0,288.0
2,4,0.043567,0.000112,Alphabet City,4,Manhattan,4461.681818,2030.363636,2431.318182,37.200000,...,205.0,0,26,431.773308,3,1.891444e+05,1398963.0,506395.0,69285.0,363.0
3,5,0.092146,0.000498,Arden Heights,5,Staten Island,5654.111111,2756.111111,2898.000000,14.744444,...,42.0,0,38,453.055556,0,3.937056e+05,35202.0,24670.0,79866.0,89.0
4,6,0.150491,0.000606,Arrochar/Fort Wadsworth,6,Staten Island,4721.292683,2282.560976,2438.731707,25.720000,...,61.0,1,90,453.055556,2,1.980973e+06,68870.0,40331.0,404014.0,162.0


## Structure Learning

In [7]:
from pgmpy.estimators import BdeuScore, K2Score, BicScore
from pgmpy.models import BayesianModel

In [8]:
data = df_spatial.copy()

In [9]:
data = data[['pickup', 'tickets', 'TotalPop', 'IncomePerCap', 'Poverty', 'Professional', 'Employed', 'crime', 'subway', 'bus', 'meter', 'parkinglot']]

In [ ]:
# data = data[['pickup', 'tickets', 'TotalPop', 'IncomePerCap']]

In [10]:
data

,pickup,tickets,TotalPop,IncomePerCap,Poverty,Professional,Employed,crime,subway,bus,meter,parkinglot
0,363.0,177889.0,528.000000,32623.000000,11.000000,31.700000,233.250000,0.0,0,4,0,1.143448e+05
1,326635.0,44411.0,4042.735294,22566.352941,17.770588,29.144118,1709.000000,288.0,3,53,16,1.170854e+06
2,1398963.0,69285.0,4461.681818,34324.772727,20.977273,40.350000,2248.227273,363.0,0,26,3,1.891444e+05
3,35202.0,79866.0,5654.111111,31403.777778,14.422222,37.233333,2531.333333,89.0,0,38,0,3.937056e+05
4,68870.0,404014.0,4721.292683,40801.575000,15.566667,44.115385,2264.000000,162.0,1,90,2,1.980973e+06
5,2481251.0,828.0,3846.102362,49926.008130,19.112195,49.183607,2045.023622,724.0,14,104,327,1.627481e+06
6,14259.0,15953.0,2774.166667,71320.400000,11.020000,60.860000,1776.166667,5.0,0,0,0,7.057751e+04
7,179981.0,126887.0,2691.650000,31420.105263,8.389474,40.868421,1276.450000,99.0,0,87,23,7.515352e+05
8,504108.0,131922.0,3175.033333,21988.750000,19.595000,25.501667,1359.466667,448.0,0,71,0,1.038856e+06
9,244751.0,1724.0,3451.087719,24997.018182,17.018182,30.378182,1571.982456,265.0,0,45,55,3.218633e+05


### ExhaustiveSearch

The search space of DAGs is super-exponential in the number of variables and the above scoring functions allow for local maxima. The first property makes exhaustive search intractable for all but very small networks, while the second property can prevent efficient local optimization algorithms from always finding the optimal structure. Thus, identifiying the ideal structure is often not tractable. Nevertheless, heuristic search strategies often yield good results.

If only few nodes are involved (read: less than 5), ExhaustiveSearch can be used to compute the score for every DAG and returns the best-scoring one.

In [ ]:
from pgmpy.estimators import ExhaustiveSearch

# Note: exhaustive search will be terribly expensive for more than a few variables
es = ExhaustiveSearch(data, scoring_method=BicScore(data))
best_model = es.estimate()
print(best_model.edges())

print("\nAll DAGs by score:")
for score, dag in reversed(es.all_scores()):
    print(score, dag.edges())

### HillClimbSearch: heuristic search
HillClimbSearch implements a greedy local search that starts from the DAG start (default: disconnected DAG) and proceeds by iteratively performing single-edge manipulations that maximally increase the score. The search terminates once a local maximum is found.

In [ ]:
from pgmpy.estimators import HillClimbSearch

# create some random data with dependencies
hc = HillClimbSearch(data, scoring_method=BicScore(data))
best_model = hc.estimate()
print(best_model.edges())

# Note: it doesn't always find the "correct" network.  To see this, try a different random seed.

### Constraint-based Structure Learning

A different approach to build a DAG from data, attempting to correctly capture the directionality of causal relationships, is this:

1. Identify independencies in the data set using hypothesis tests 
2. Construct a DAG according to identified independencies

#### (Conditional) Independence Tests

Independencies in the data can be identified using $\chi^2$ conditional independence tests. To this end, constraint-based estimators in pgmpy have a `test_conditional_independence(X, Y, Zs)`-method, that performs a hypothesis test on the data sample. It allows to check if `X` is independent from `Y` given a set of variables `Zs`:

In [ ]:
from pgmpy.estimators import ConstraintBasedEstimator

In [ ]:
est = ConstraintBasedEstimator(data)

In [ ]:
print(est.test_conditional_independence('pickup', 'tickets'))          # independent

In [ ]:
print(est.test_conditional_independence('B', 'H'))          # dependent
print(est.test_conditional_independence('B', 'H', ['A']))   # independent
print(est.test_conditional_independence('A', 'G',  ['H']))  # dependent

In [ ]:
def is_independent(X, Y, Zs=[], significance_level=0.05):
    return est.test_conditional_independence(X, Y, Zs)[1] >= significance_level

In [ ]:
print(is_independent('B', 'C'))
print(is_independent('B', 'H'))
print(is_independent('B', 'E'))
print(is_independent('B', 'H', ['A']))
print(is_independent('A', 'G'))
print(is_independent('A', 'G', ['H']))

#### DAG (pattern) construction

#### PC Algorithm for causal direction

With a method for independence testing at hand, we can construct a DAG from the data set in three steps:

*1. Construct an undirected skeleton - `estimate_skeleton()`

*2. Orient compelled edges to obtain partially directed acyclic graph (PDAG; I-equivalence class of DAGs) - `skeleton_to_pdag()`

Steps 1 and 2 form the PC algorithm. PDAGs are `DirectedGraph`s, that may contain bidirectional edges, to indicate that the orientation for the edge is not determined.

####  The following step orients any remaining edges, essentially at random, but in a way which is consistent with the edge directions produced by PC.  This is useful if we want to use the learned structure to perform inference, but we should not make any assumptions re. causality from the edges directed in the last step.

 *3. Extend DAG pattern to a DAG by orienting the remaining edges in some consistent way - `pdag_to_dag()`


In [11]:
from pgmpy.estimators import ConstraintBasedEstimator

In [12]:
est = ConstraintBasedEstimator(data)

In [13]:
skel, separating_sets = est.estimate_skeleton(significance_level=0.01)
print("Undirected edges: ", skel.edges())

pdag = est.skeleton_to_pdag(skel, separating_sets)
print("PDAG edges:       ", pdag.edges())

model = est.pdag_to_dag(pdag)
print("DAG edges:        ", model.edges())

C:\ProgramData\Anaconda3\lib\site-packages\pgmpy\estimators\base.py:209: UserWarning: Insufficient data for testing pickup _|_ tickets | []. At least 330245.0 samples recommended, 258 present.
  "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))
C:\ProgramData\Anaconda3\lib\site-packages\pgmpy\estimators\base.py:209: UserWarning: Insufficient data for testing pickup _|_ TotalPop | []. At least 328960.0 samples recommended, 258 present.
  "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))
C:\ProgramData\Anaconda3\lib\site-packages\pgmpy\estimators\base.py:209: UserWarning: Insufficient data for testing pickup _|_ IncomePerCap | []. At least 327675.0 samples recommended, 258 present.
  "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))
C:\ProgramData\Anaconda3\lib\site-packages\pgmpy\estimators\base.py:209: UserWarning: Insufficient data for testing pickup _|_ Poverty | []. A

C:\ProgramData\Anaconda3\lib\site-packages\pgmpy\estimators\base.py:209: UserWarning: Insufficient data for testing TotalPop _|_ parkinglot | []. At least 318720.0 samples recommended, 258 present.
  "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))
C:\ProgramData\Anaconda3\lib\site-packages\pgmpy\estimators\base.py:209: UserWarning: Insufficient data for testing IncomePerCap _|_ Poverty | []. At least 323850.0 samples recommended, 258 present.
  "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))
C:\ProgramData\Anaconda3\lib\site-packages\pgmpy\estimators\base.py:209: UserWarning: Insufficient data for testing IncomePerCap _|_ Professional | []. At least 323850.0 samples recommended, 258 present.
  "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))
C:\ProgramData\Anaconda3\lib\site-packages\pgmpy\estimators\base.py:209: UserWarning: Insufficient data for testing IncomePerC

C:\ProgramData\Anaconda3\lib\site-packages\pgmpy\estimators\base.py:209: UserWarning: Insufficient data for testing crime _|_ meter | []. At least 137020.0 samples recommended, 258 present.
  "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))
C:\ProgramData\Anaconda3\lib\site-packages\pgmpy\estimators\base.py:209: UserWarning: Insufficient data for testing crime _|_ parkinglot | []. At least 275145.0 samples recommended, 258 present.
  "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))
C:\ProgramData\Anaconda3\lib\site-packages\pgmpy\estimators\base.py:209: UserWarning: Insufficient data for testing subway _|_ crime | []. At least 37570.0 samples recommended, 258 present.
  "At least {0} samples recommended, {1} present.".format(5 * num_params, len(self.data)))
C:\ProgramData\Anaconda3\lib\site-packages\pgmpy\estimators\base.py:209: UserWarning: Insufficient data for testing subway _|_ bus | []. At least 21930.

Undirected edges:  [('subway', 'meter')]
PDAG edges:        [('subway', 'meter'), ('meter', 'subway')]
DAG edges:         [('meter', 'subway')]


## Parameter Learning
Given a set of data records and a DAG that captures the dependencies between the variables, estimate the (conditional) probability distributions of the individual variables.


In [8]:
data = df_spatial.copy()
data

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,TotalPop,Men,Women,Hispanic,...,MISDEMEANOR,subway,bus,sat,meter,parkinglot,pickup,dropoff,tickets,crime
0,2,0.433470,0.004866,Jamaica Bay,2,Queens,528.000000,248.750000,279.250000,4.900000,...,0.0,0,4,382.578884,0,1.143448e+05,363.0,211.0,177889.0,0.0
1,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,4042.735294,1848.558824,2194.176471,24.332353,...,173.0,3,53,385.980392,16,1.170854e+06,326635.0,220261.0,44411.0,288.0
2,4,0.043567,0.000112,Alphabet City,4,Manhattan,4461.681818,2030.363636,2431.318182,37.200000,...,205.0,0,26,431.773308,3,1.891444e+05,1398963.0,506395.0,69285.0,363.0
3,5,0.092146,0.000498,Arden Heights,5,Staten Island,5654.111111,2756.111111,2898.000000,14.744444,...,42.0,0,38,453.055556,0,3.937056e+05,35202.0,24670.0,79866.0,89.0
4,6,0.150491,0.000606,Arrochar/Fort Wadsworth,6,Staten Island,4721.292683,2282.560976,2438.731707,25.720000,...,61.0,1,90,453.055556,2,1.980973e+06,68870.0,40331.0,404014.0,162.0
5,7,0.107417,0.000390,Astoria,7,Queens,3846.102362,1851.574803,1994.527559,28.482258,...,421.0,14,104,446.416667,327,1.627481e+06,2481251.0,1273753.0,828.0,724.0
6,8,0.027591,0.000027,Astoria Park,8,Queens,2774.166667,1434.333333,1339.833333,9.420000,...,2.0,0,0,446.416667,0,7.057751e+04,14259.0,9436.0,15953.0,5.0
7,9,0.099784,0.000338,Auburndale,9,Queens,2691.650000,1327.500000,1364.150000,14.084211,...,46.0,0,87,446.206061,23,7.515352e+05,179981.0,108402.0,126887.0,99.0
8,10,0.099839,0.000436,Baisley Park,10,Queens,3175.033333,1513.700000,1661.333333,19.466667,...,204.0,0,71,425.469697,0,1.038856e+06,504108.0,270745.0,131922.0,448.0
9,11,0.079211,0.000265,Bath Beach,11,Brooklyn,3451.087719,1640.456140,1810.631579,22.134545,...,152.0,0,45,418.400000,55,3.218633e+05,244751.0,158043.0,1724.0,265.0


In [9]:
data.columns

Index(['OBJECTID', 'Shape_Leng', 'Shape_Area', 'zone', 'LocationID', 'borough',
       'TotalPop', 'Men', 'Women', 'Hispanic', 'White', 'Black', 'Native',
       'Asian', 'Citizen', 'Income', 'IncomeErr', 'IncomePerCap',
       'IncomePerCapErr', 'Poverty', 'ChildPoverty', 'Professional', 'Service',
       'Office', 'Construction', 'Production', 'Drive', 'Carpool', 'Transit',
       'Walk', 'OtherTransp', 'WorkAtHome', 'MeanCommute', 'Employed',
       'PrivateWork', 'PublicWork', 'SelfEmployed', 'FamilyWork',
       'Unemployment', 'FELONY', 'VIOLATION', 'MISDEMEANOR', 'subway', 'bus',
       'sat', 'meter', 'parkinglot', 'pickup', 'dropoff', 'tickets', 'crime'],
      dtype='object')

In [10]:
from pgmpy.models import BayesianModel
model = BayesianModel([('tickets', 'pickup'), ('Carpool', 'pickup')])

In [11]:
from pgmpy.models import BayesianModel
model = BayesianModel([
                      ('Professional', 'IncomePerCap'), \
                      ('sat', 'IncomePerCap'), ("IncomePerCap", "Poverty"), ("IncomePerCap", "Carpool"), ("Poverty", "MISDEMEANOR"), ("MISDEMEANOR", "tickets"),\
                      ("Carpool", "pickup"), ("pickup", "tickets"),("meter", "tickets"),\
                      ("TotalPop", "Carpool"), ("subway", "pickup"), ("bus", "pickup"), ("parkinglot", "tickets"),\
                      ("TotalPop", "pickup"), ("TotalPop", "bus"),("TotalPop", "parkinglot")\
                      ])

Parameter learning is the task of estimating the values of the conditional probability distributions (CPDs), for the variables.

#### State counts

from pgmpy.estimators import ParameterEstimator
pe = ParameterEstimator(model, data)
print(pe.state_counts('tickets'))  # unconditional
print()
print(pe.state_counts('pickup'))  # conditional on fruit and size

#### Maximum Likelihood Estimation (MLE). 
According to MLE, we should fill the CPDs in such a way, that $P(\text{data}\:|\:\text{model})$ is maximal.

In [12]:
from pgmpy.estimators import MaximumLikelihoodEstimator
mle = MaximumLikelihoodEstimator(model, data)

In [13]:
print(mle.estimate_cpd('pickup'))

MemoryError: 

In [ ]:
print(mle.estimate_cpd('tickets'))

# Temporal

In [22]:
temporal = pd.read_csv('output_final/temporal.csv')
print(temporal.shape)
temporal.head()

(1277, 9)


,date,tickets,WindSpeed,Precipitation,Snow,Temperature,holidays,weekdays,pickup
0,2015-01-01,5375.0,7.16,0.00,0.0,33.0,0,1,48715.0
1,2015-01-02,47233.0,7.16,0.00,0.0,38.5,1,0,33877.0
2,2015-01-03,23933.0,6.49,0.71,0.0,37.5,0,1,50228.0
3,2015-01-04,6651.0,6.49,0.30,0.0,48.5,0,0,34961.0
4,2015-01-05,46178.0,10.51,0.00,0.0,35.0,0,0,36290.0


## Dynamic Bayesian network
reference:   
https://support.bayesfusion.com/forum/viewtopic.php?t=4621  
https://github.com/topics/dynamic-bayesian-networks  
https://github.com/daanknoope/DBN_learner